In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np 
import cv2 as cv

import zipfile
from io import BytesIO
from PIL import Image

import gc
from dataAugmentationLib import placeImagesOnBackground, readImagesFromDirectories
from keras.models import load_model, clone_model
from sklearn.model_selection import train_test_split

In [2]:
tf.config.set_visible_devices([], 'GPU')

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
IMAGE_SHAPE = (200, 272) # int(1/(1.37/400))
IMAGE_SHAPE_OUT = (200, 272) # ae

In [5]:
background_img = cv.imread("./data/blank.jpg") - 24
wheat_images = readImagesFromDirectories("./data/objects/wheat/seeds")
misc_images = readImagesFromDirectories("./data/objects/wheat/miscs")

In [6]:
x_train, y_train = [], []
x_test, y_test = [], []

In [7]:
def load_images_from_zip(zip_file_path, x_train):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
        print(zip_file.namelist())
        length = len(zip_file.namelist())
        print(zip_file_path, length)
        for file_name in zip_file.namelist()[0:100]:
            # Filter files with image extensions (you can add more extensions as needed)
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                with zip_file.open(file_name) as image_data:
                    image_bytes = BytesIO(image_data.read())
                    image = np.array(Image.open(image_bytes))
                    
                    x_train.append(image)

In [8]:
load_images_from_zip('./data/augmented/image.zip', x_train)
load_images_from_zip('./data/augmented/mask.zip', y_train)


['image/', 'image/0.jpg', 'image/1.jpg', 'image/10.jpg', 'image/100.jpg', 'image/1000.jpg', 'image/1001.jpg', 'image/1002.jpg', 'image/1003.jpg', 'image/1004.jpg', 'image/1005.jpg', 'image/1006.jpg', 'image/1007.jpg', 'image/1008.jpg', 'image/1009.jpg', 'image/101.jpg', 'image/1010.jpg', 'image/1011.jpg', 'image/1012.jpg', 'image/1013.jpg', 'image/1014.jpg', 'image/1015.jpg', 'image/1016.jpg', 'image/1017.jpg', 'image/1018.jpg', 'image/1019.jpg', 'image/102.jpg', 'image/1020.jpg', 'image/1021.jpg', 'image/1022.jpg', 'image/1023.jpg', 'image/1024.jpg', 'image/1025.jpg', 'image/1026.jpg', 'image/1027.jpg', 'image/1028.jpg', 'image/1029.jpg', 'image/103.jpg', 'image/1030.jpg', 'image/1031.jpg', 'image/1032.jpg', 'image/1033.jpg', 'image/1034.jpg', 'image/1035.jpg', 'image/1036.jpg', 'image/1037.jpg', 'image/1038.jpg', 'image/1039.jpg', 'image/104.jpg', 'image/1040.jpg', 'image/1041.jpg', 'image/1042.jpg', 'image/1043.jpg', 'image/1044.jpg', 'image/1045.jpg', 'image/1046.jpg', 'image/1047.

each 0.15 secs

In [ ]:
for i in range(10000):
    augmentedImage, augmentedMask = placeImagesOnBackground(background_img, wheat_images, misc_images)
    cv.imwrite(f'./data/augmented/image/{i}.jpg', cv.resize(augmentedImage, IMAGE_SHAPE))
    cv.imwrite(f'./data/augmented/mask/{i}.jpg', cv.resize(augmentedMask, IMAGE_SHAPE))
    if i%1000==0:
        print(i, " images saved")
    # x_train.append(cv.resize(augmentedImage, IMAGE_SHAPE))
    # y_train.append(cv.resize(augmentedMask, IMAGE_SHAPE_OUT))

In [9]:
del wheat_images, misc_images

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

x

In [11]:
x_train = np.array(x_train).astype('float64')
y_train = np.array(y_train).astype('float64')
# y_train = np.squeeze(y_train)

x_test = np.array(x_test).astype('float64')
y_test = np.array(y_test).astype('float64')

In [12]:
gc.collect()

0

In [13]:
x_train = x_train/255
x_test = x_test/255
y_train = y_train/255
y_test = y_test/255

In [14]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(79, 272, 200, 3)
(20, 272, 200, 3)
(79, 272, 200)
(20, 272, 200)


In [20]:
for i in range(10000):
    cv.imshow('x_train', x_train[i])
    cv.imshow('y_train', y_train[i])
    key = cv.waitKey(500000)
    if key==113: #q
        break
cv.destroyAllWindows()

In [15]:
conv_ae = load_model('model.h5')

In [16]:
for i in range(10000):
    # p = conv_ae.predict(np.array([cv.resize(image, IMAGE_SHAPE)/255]))
    p = conv_ae.predict(x_train[i:i+1])
    red = p[0].copy()
    red[red>=0.75] = 0
    red[red<=0.25] = 0
    red[np.logical_and(red > 0.25, red < 0.75)] = 1
    red = np.reshape(red, (272, 200))

    green = p[0].copy()
    green[green>=0.75] = 1
    green[green<=0.25] = 0
    green[np.logical_and(green > 0.25, green < 0.75)] = 0
    green = np.reshape(green, (272, 200))

    tempImage = cv.resize(x_train[i], IMAGE_SHAPE_OUT)
    tempImage[:, :, 1] = np.where(green == 1, 1, tempImage[:, :, 1])
    tempImage[:, :, 2] = np.where(red == 1, 1, tempImage[:, :, 2])

    cv.imshow('p', p[0])
    cv.imshow('red', red)
    cv.imshow('green', green)
    cv.imshow('tempImage', cv.resize(tempImage, (745, 1024)))
    cv.imshow('x_train', x_train[i])
    cv.imshow('y_train', y_train[i])
    # cv.imshow('x_test', x_test[i])
    # cv.imshow('y_test', y_test[i])
    key = cv.waitKey(500000)
    if key==113: #q
        break
cv.destroyAllWindows()

1/1 [==============================] - 0s 25ms/step


In [ ]:
image = cv.imread('./data/2_1.jpg')
for i in range(10000):
    p = conv_ae.predict(np.array([cv.resize(image, IMAGE_SHAPE)/255]))
    # p = conv_ae.predict(x_test[i:i+1])

    red = p[0].copy()
    red[red>=0.75] = 0
    red[red<=0.25] = 0
    red[np.logical_and(red > 0.25, red < 0.75)] = 1
    red = np.reshape(red, (272, 200))

    green = p[0].copy()
    green[green>=0.75] = 1
    green[green<=0.25] = 0
    green[np.logical_and(green > 0.25, green < 0.75)] = 0
    green = np.reshape(green, (272, 200))

    # tempImage = cv.resize(np.float32(x_test[i]), IMAGE_SHAPE_OUT)
    # tempImage[:, :, 1] = np.where(green == 1, 1, tempImage[:, :, 1])
    # tempImage[:, :, 2] = np.where(red == 1, 1, tempImage[:, :, 2])

    cv.imshow('p', p[0])
    cv.imshow('red', red)
    cv.imshow('green', green)
    # cv.imshow('tempImage', cv.resize(tempImage, (745, 1024)))
    # cv.imshow('x_test', np.float32(x_test[i]))
    # cv.imshow('y_test', np.float32(y_test[i]))
    key = cv.waitKey(500000)
    if key==113: #q
        break
cv.destroyAllWindows()